In [2]:
## Importing libraries 
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
import numpy as np
import soundfile as sf
from pathlib import Path
from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator
from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.models import FastPitchModel
import IPython.display as ipd

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-07-18 10:24:12 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-18 10:24:12 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [47]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager
from scipy.io import wavfile
import noisereduce as nr

In [4]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-07-18 10:24:13 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-07-18 10:24:13 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-07-18 10:24:13 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-18 10:24:14 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-07-18 10:24:14 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-07-18 10:24:14 features:200] PADDING: 16
[NeMo I 2022-07-18 10:24:18 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


# Finetuning FastPitch and testing original WER

In [205]:
# For original audio training
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-12 17:20:10 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-12 17:20:11 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-12 17:20:12 exp_manager:287] Experiments will be logged at


Epoch 24: 100%|██████████| 26/26 [00:21<00:00,  1.19it/s, loss=4.18, v_num=0-12]Traceback (most recent call last):
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/queues.py", line 235, in _feed
    close()
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Epoch 24: 100%|██████████| 26/26 [00:22<00:00,  1.14it/s, loss=4.18, v_num=0-12]
                                                                                Epoch 24, global step 625: 'v_loss' reached 1.19441 (best 1.19441), saving model to '/home/dgxuser/Desktop/kailin/S91_no_mixing/FastPitch/2022-07-12_17-20-12/checkpoints/FastPitch--v_loss=1.1944-epoch=24.ckpt' as top 3
Epoch 39: 100%|██████████| 25/25 [00:13<00:00,  1.88it/s, loss=4.08, v_num=0-12]


In [22]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [23]:
speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

S91_no_mixing/FastPitch/2022-07-12_17-20-12/checkpoints/FastPitch--v_loss=1.1944-epoch=24-last.ckpt
[NeMo I 2022-07-18 11:10:57 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-18 11:11:26 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-18 11:11:26 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_d

[NeMo I 2022-07-18 11:11:26 features:200] PADDING: 1
[NeMo I 2022-07-18 11:11:26 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-18 11:11:26 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-18 11:11:26 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-18 11:11:30 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-18 11:11:30 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-18 11:11:30 features:200] PADDING: 0


[NeMo W 2022-07-18 11:11:30 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-18 11:11:30 features:200] PADDING: 0
[NeMo I 2022-07-18 11:11:32 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [24]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_original/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_original.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [25]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 0.2213328693231251


# FineTuning of HiFiGan

In [14]:
def load_wav(audio_file, target_sr=None):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
        sample_rate = f.samplerate
        if target_sr is not None and target_sr != sample_rate:
            samples = librosa.core.resample(samples, orig_sr=sample_rate, target_sr=target_sr)
    return samples.transpose()

# Get records from the training manifest
records = []
train_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
with open(train_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect/S91/trainSpect')
save_dir.mkdir(exist_ok=True, parents=True)

print(records)
# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    print(r)
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        #if "normalized_text" in r:
            #text = spec_model.parse(r["normalized_text"], normalize=False)
        #else:
            #text = spec_model.parse(r['text'])
        
        text = spec_model.parse(r['text'])
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

[{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910001.WAV', 'text': 'mary and her family were moving to another city', 'duration': 3.72}, {'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910002.WAV', 'text': 'with the movers hired to settle the rest of their luggage and furniture they decided to take a short tour first of the harbour that was near their new home', 'duration': 8.97}, {'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910003.WAV', 'text': "they brought some clothes and three days' worth of food and water in their car", 'duration': 4.66}, {'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910005.WAV', 'text': 'enthusiastically they got out of the car and took a lot of pictures as they hiked up a gentle slope towards the top of the hill', 'duration': 7.99}, {'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAK

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910002.WAV', 'text': 'with the movers hired to settle the rest of their luggage and furniture they decided to take a short tour first of the harbour that was near their new home', 'duration': 8.97}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910003.WAV', 'text': "they brought some clothes and three days' worth of food and water in their car", 'duration': 4.66}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910005.WAV', 'text': 'enthusiastically they got out of the car and took a lot of pictures as they hiked up a gentle slope towards the top of the hill', 'duration': 7.99}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910006.WAV', 'text': 'when they reached they saw a huge tree and sat beneath the foliage', 'duration': 4.61}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANN

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910053.WAV', 'text': 'tell him to zip his lip', 'duration': 2.461}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910054.WAV', 'text': 'i like listening to rap', 'duration': 2.425}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910057.WAV', 'text': 'the witch put a spell on the prince but it was the wrong one', 'duration': 4.3}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910058.WAV', 'text': 'a murder had already been plotted a month ago', 'duration': 3.83}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910059.WAV', 'text': 'i had a lot of questions about the assignment', 'duration': 3.31}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910060.WAV', 'text': 'hold your bat steady and focus your eyes on 

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910109.WAV', 'text': 'as a matter of principle we should pay according to the value of the job', 'duration': 4.31}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910110.WAV', 'text': 'some were up even before the crack of dawn', 'duration': 3.34}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910111.WAV', 'text': "they then approached mister ong's friends and colleagues to put the book together", 'duration': 4.83}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910112.WAV', 'text': 'during the early formation years all of us went through a period of hardship and turmoil', 'duration': 5.64}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION0/000910113.WAV', 'text': 'a fruitful and enjoyable retirement requires early planning', 'duration': 4.26}
{'audio_

...

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911517.WAV', 'text': 'the public service is also guided by a code of conduct that details principles and rules they must abide by', 'duration': 5.96}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911518.WAV', 'text': 'an effective free market entails that there are enough players who will be able to compete with one another', 'duration': 6.21}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911519.WAV', 'text': 'the footage from inside the lift also showed the interaction between them', 'duration': 4.698}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911521.WAV', 'text': 'oil may also soon get some support from seasonal demand', 'duration': 4.54}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911522.WAV', 'text': 'research has shown that ther

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911570.WAV', 'text': 'but he is now uses a wheelchair and has and has undergone three chemotherapy sessions and multiple blood transfusions', 'duration': 8.259}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911571.WAV', 'text': 'a torn mattress wrapped in canvas sheets sits in the middle', 'duration': 4.479}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911574.WAV', 'text': "i've really come to become very fond of them and i will miss them dearly", 'duration': 4.96}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911575.WAV', 'text': 'but exact timing has not been determined', 'duration': 3.244}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911577.WAV', 'text': 'more details can be expected by the end of the year', 'duration': 3.58}
{'audio_

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911631.WAV', 'text': 'but they will first need to pick up the necessary skills to use such standardised software', 'duration': 5.63}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911632.WAV', 'text': 'if it does catch fire the bag will help contain it before it can spread', 'duration': 4.69}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911633.WAV', 'text': 'experts today also suggested that local teachers be trained differently', 'duration': 4.72}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911634.WAV', 'text': 'here take this video to jog your memory', 'duration': 3.464}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911635.WAV', 'text': 'going for a reform party rally', 'duration': 2.76}
{'audio_filepath': '/home/dgxuser/Desktop/kailin

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911683.WAV', 'text': 'as soon as they could feel his pulse i was relieved', 'duration': 3.853}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911684.WAV', 'text': 'u s authorities should not base government decisions on speculation or rumour', 'duration': 5.58}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911686.WAV', 'text': 'dealing with the emotions that come with being diagnosed with breast cancer is not an easy one', 'duration': 5.986}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911687.WAV', 'text': 'but the hysterical woman kicked her in the chest leaving her winded and shaken for the rest of the day', 'duration': 5.972}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911688.WAV', 'text': 'besides driving economic stability businesse

{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911734.WAV', 'text': 'the annual award is the highest accolade in the profession recognising outstanding nurses from both the public and private sectors', 'duration': 7.39}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911735.WAV', 'text': 'it means that the dining experience you get at the restaurant is worth more than what you pay for', 'duration': 5.082}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911736.WAV', 'text': "parents tell us that's what they come to us for", 'duration': 4.196}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911737.WAV', 'text': 'clearly he was taking inspiration from his own name', 'duration': 3.72}
{'audio_filepath': '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE/SPEAKER0091/SESSION1/000911738.WAV', 'text': 'better to get your agent to go through with 

In [16]:
# Get records from the validation manifest
records = []
val_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
with open(val_manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

beta_binomial_interpolator = BetaBinomialInterpolator()
spec_model.eval()

device = spec_model.device

save_dir = Path('/home/dgxuser/Desktop/kailin/melSpect/S91/valSpect')
save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    audio = load_wav(r["audio_filepath"])
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    
    # Again, our finetuned FastPitch model doesn't use multiple speakers,
    # but we keep the code to support it here for reference
    if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
        speaker = torch.tensor([r['speaker']]).to(device)
    else:
        speaker = None
    
    with torch.no_grad():
        #if "normalized_text" in r:
            #text = spec_model.parse(r["normalized_text"], normalize=False)
        #else:
            #text = spec_model.parse(r['text'])
            
        text = spec_model.parse(r['text'])
        
        text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    
        spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

        # Generate attention prior and spectrogram inputs for HiFi-GAN
        attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
            
        spectrogram = spec_model.forward(
          text=text, 
          input_lens=text_len, 
          spec=spect, 
          mel_lens=spect_len, 
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
        
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_val.json'
with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')

In [17]:
!(python hifigan_finetune.py \
  --config-name=hifigan.yaml \
  model.train_ds.dataloader_params.batch_size=32 \
  model.max_steps=1000 \
  ~model.optim.sched \
  model.optim.lr=0.00001 \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_val.json \
  exp_manager.exp_dir=S91_hifigan_ft \
  +init_from_pretrained_model=tts_hifigan \
  trainer.check_val_every_n_epoch=10 \
  model/train_ds=train_ds_finetune \
  model/validation_ds=val_ds_finetune)

[NeMo W 2022-07-18 10:57:31 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-18 10:57:32 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-07-18 10:57:32 nemo_logging:349] /home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'hifigan.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_co

[NeMo W 2022-07-18 10:57:45 nemo_logging:349] /home/dgxuser/anaconda3/envs/test4/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (19) is smaller than the logging interval Trainer(log_every_n_steps=100). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
      rank_zero_warn(
    
Epoch 0:   0%|                                           | 0/19 [00:00<?, ?it/s][W reducer.cpp:1289] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator(

In [18]:
def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        model_name="HifiGan"
    ):    
    
    d = f"S{new_speaker_id}_hifigan_ft"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [19]:
speaker_id = 91
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id)
print(last_ckpt)
vocoder = HifiGanModel.load_from_checkpoint(last_ckpt)
vocoder = vocoder.eval().cuda()

S91_hifigan_ft/HifiGan/2022-07-18_10-57-32/checkpoints/HifiGan--val_loss=1.2126-epoch=19-last.ckpt


[NeMo W 2022-07-18 11:09:54 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/hifigan_S91_train.json
      sample_rate: 22050
      n_segments: 8192
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 256
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 32
      num_workers: 4
      pin_memory: true
    
[NeMo W 2022-07-18 11:09:54 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collecti

[NeMo I 2022-07-18 11:09:54 features:200] PADDING: 0
[NeMo I 2022-07-18 11:09:54 features:208] STFT using exact pad
[NeMo I 2022-07-18 11:09:54 features:200] PADDING: 0
[NeMo I 2022-07-18 11:09:54 features:208] STFT using exact pad


In [20]:
from pydub import AudioSegment
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_ft/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_ft.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [223]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [224]:
error = wer(transcript, predicted)
print('Word Error Rate (FineTuned): ' + str(error))

Word Error Rate (FineTuned): 0.822690099182182


# Downsampling to 16kHz before transcription by Quartznet

In [188]:
vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

[NeMo I 2022-07-12 15:58:36 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-12 15:58:36 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-12 15:58:36 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-12 15:58:39 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-12 15:58:39 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-12 15:58:39 features:200] PADDING: 0


[NeMo W 2022-07-12 15:58:39 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-12 15:58:39 features:200] PADDING: 0
[NeMo I 2022-07-12 15:58:41 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [189]:
from pydub import AudioSegment
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_resampled/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_resampled.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")
    
      # resample at 16kHz
      sound = AudioSegment.from_file(file_dir, format='wav', frame_rate=22050)
      sound = sound.set_frame_rate(16000)
      file_dir = audio_dir + str(n) + '_resampled.wav'
      sound.export(file_dir, format='wav')

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [190]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [191]:
error = wer(transcript, predicted)
print('Word Error Rate (Resampled): ' + str(error))

Word Error Rate (Resampled): 0.21211066643466156


# Upsampling audio to 22.05kHz before passing in to Fastpitch

In [28]:
from pydub import AudioSegment
audio_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/UPSAMPLED_WAVE/'
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
new_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_upsampled_train.json'

n = 0
with open(new_manifest_path, 'w') as fout: 
    with open(manifest_path, 'r') as fin: 
        line = fin.readline()
        while line != '': 
            data = json.loads(line)
            file_dir = data["audio_filepath"]
            
            # resample at 22.05kHz
            sound = AudioSegment.from_file(file_dir, format='wav', frame_rate=22050)
            sound = sound.set_frame_rate(22050)
            file_dir = audio_dir + str(n) + '.wav'
            sound.export(file_dir, format='wav')         
            
            metadata = {
            "audio_filepath": file_dir,
            "duration": data["duration"], 
            "text": data["text"]
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            line = fin.readline()
            n = n + 1 

In [30]:
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
new_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_upsampled_val.json'

with open(new_manifest_path, 'w') as fout: 
    with open(manifest_path, 'r') as fin: 
        line = fin.readline()
        while line != '': 
            data = json.loads(line)
            file_dir = data["audio_filepath"]
            
            # resample at 22.05kHz
            sound = AudioSegment.from_file(file_dir, format='wav', frame_rate=22050)
            sound = sound.set_frame_rate(22050)
            file_dir = audio_dir + str(n) + '.wav'
            sound.export(file_dir, format='wav')         
            
            metadata = {
            "audio_filepath": file_dir,
            "duration": data["duration"], 
            "text": data["text"]
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            line = fin.readline()
            n = n + 1 

In [31]:
# For original audio training
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_upsampled_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_upsampled_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-18 11:29:16 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-18 11:29:17 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-18 11:29:18 exp_manager:287] Experiments will be logged at

Epoch 39: 100%|██████████| 25/25 [00:08<00:00,  3.12it/s, loss=4.08, v_num=9-18]


In [33]:
def infer(spec_gen_model, vocoder_model, str_input):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=None)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        mixing_enabled, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"S{new_speaker_id}_{mixing}"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [34]:
speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

S91_no_mixing/FastPitch/2022-07-18_11-29-18/checkpoints/FastPitch--v_loss=1.1467-epoch=24-last.ckpt
[NeMo I 2022-07-18 11:39:19 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-18 11:39:50 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-18 11:39:50 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_upsampled_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
 

[NeMo I 2022-07-18 11:39:50 features:200] PADDING: 1
[NeMo I 2022-07-18 11:39:51 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-18 11:39:51 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-18 11:39:51 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-18 11:39:54 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-18 11:39:54 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-18 11:39:54 features:200] PADDING: 0


[NeMo W 2022-07-18 11:39:54 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-18 11:39:54 features:200] PADDING: 0
[NeMo I 2022-07-18 11:39:56 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [35]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_upsampled/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_upsampled.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [36]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
error = wer(transcript, predicted)
print('Word Error Rate (FineTuned): ' + str(error))

Word Error Rate (FineTuned): 0.21646076213676702


# Remove silence at front and back of audio

In [38]:
import soundfile as sf
audio_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SILENCE_WAVE/'
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
new_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_silence_train.json'

n = 0
with open(new_manifest_path, 'w') as fout: 
    with open(manifest_path, 'r') as fin: 
        line = fin.readline()
        while line != '': 
            data = json.loads(line)
            file_dir = data["audio_filepath"]
            
            audio, sr = librosa.load(file_dir)
            clip = librosa.effects.trim(audio, top_db=10)
            file_dir = audio_dir + str(n) + '.wav'
            sf.write(file_dir, clip[0], sr)

            metadata = {
            "audio_filepath": file_dir,
            "duration": data["duration"], 
            "text": data["text"]
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            line = fin.readline()
            n = n + 1 

In [39]:
audio_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SILENCE_WAVE/'
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
new_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_silence_val.json'

with open(new_manifest_path, 'w') as fout: 
    with open(manifest_path, 'r') as fin: 
        line = fin.readline()
        while line != '': 
            data = json.loads(line)
            file_dir = data["audio_filepath"]
            
            audio, sr = librosa.load(file_dir)
            clip = librosa.effects.trim(audio, top_db=10)
            file_dir = audio_dir + str(n) + '.wav'
            sf.write(file_dir, clip[0], sr)

            metadata = {
            "audio_filepath": file_dir,
            "duration": data["duration"], 
            "text": data["text"]
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            line = fin.readline()
            n = n + 1 

In [41]:
# For original audio training
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_silence_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_silence_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-18 14:24:54 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-18 14:24:55 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-18 14:24:56 exp_manager:287] Experiments will be logged at

                                                                                Epoch 24, global step 625: 'v_loss' reached 1.33661 (best 1.33661), saving model to '/home/dgxuser/Desktop/kailin/S91_no_mixing/FastPitch/2022-07-18_14-24-56/checkpoints/FastPitch--v_loss=1.3366-epoch=24.ckpt' as top 3
Epoch 39: 100%|██████████| 25/25 [00:06<00:00,  3.64it/s, loss=3.13, v_num=4-56]


In [42]:
speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

S91_no_mixing/FastPitch/2022-07-18_14-24-56/checkpoints/FastPitch--v_loss=1.3366-epoch=24-last.ckpt
[NeMo I 2022-07-18 14:31:20 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-18 14:31:49 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-18 14:31:49 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_silence_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
   

[NeMo I 2022-07-18 14:31:49 features:200] PADDING: 1
[NeMo I 2022-07-18 14:31:50 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-18 14:31:50 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-18 14:31:50 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-18 14:31:53 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-18 14:31:53 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-18 14:31:53 features:200] PADDING: 0


[NeMo W 2022-07-18 14:31:53 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-18 14:31:53 features:200] PADDING: 0
[NeMo I 2022-07-18 14:31:55 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [43]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_silence/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_silence.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [45]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [46]:
error = wer(transcript, predicted)
print('Word Error Rate (FineTuned): ' + str(error))

Word Error Rate (FineTuned): 0.1961023142509135


# Cleaning audio before transcription by Quartznet

In [48]:
from pydub import AudioSegment
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_cleanAfter/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_cleanAfter.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")
    
      rate, data = wavfile.read(file_dir)
      # perform noise reduction
      reduced_noise = nr.reduce_noise(y=data, sr=rate)
      file_dir = audio_dir + str(n) + '.wav'
      wavfile.write(file_dir, rate, reduced_noise)

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [49]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
error = wer(transcript, predicted)
print('Word Error Rate (FineTuned): ' + str(error))

Word Error Rate (FineTuned): 0.23159909518009397


# Cleaning audio before passing in to Fastpitch

In [52]:
audio_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/CLEAN_WAVE/'
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_train.json'
new_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_clean_train.json'

n = 0
with open(new_manifest_path, 'w') as fout: 
    with open(manifest_path, 'r') as fin: 
        line = fin.readline()
        while line != '': 
            data = json.loads(line)
            file_dir = data["audio_filepath"]
            
            rate, audio = wavfile.read(file_dir)
            # perform noise reduction
            reduced_noise = nr.reduce_noise(y=audio, sr=rate)
            file_dir = audio_dir + str(n) + '.wav'
            wavfile.write(file_dir, rate, reduced_noise)
            
            metadata = {
            "audio_filepath": file_dir,
            "duration": data["duration"], 
            "text": data["text"]
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            line = fin.readline()
            n = n + 1 

In [53]:
audio_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/CLEAN_WAVE/'
manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_val.json'
new_manifest_path = '/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_clean_val.json'

with open(new_manifest_path, 'w') as fout: 
    with open(manifest_path, 'r') as fin: 
        line = fin.readline()
        while line != '': 
            data = json.loads(line)
            file_dir = data["audio_filepath"]
            
            rate, audio = wavfile.read(file_dir)
            # perform noise reduction
            reduced_noise = nr.reduce_noise(y=audio, sr=rate)
            file_dir = audio_dir + str(n) + '.wav'
            wavfile.write(file_dir, rate, reduced_noise)
            
            metadata = {
            "audio_filepath": file_dir,
            "duration": data["duration"], 
            "text": data["text"]
            }
            json.dump(metadata, fout)
            fout.write('\n')   
            line = fin.readline()
            n = n + 1 

In [54]:
# For original audio training
!(python fastpitch_finetune.py --config-name=fastpitch_align_v1.05.yaml \
  train_dataset=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_clean_train.json \
  validation_datasets=/home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_clean_val.json \
  sup_data_path=./fastpitch_sup_data \
  phoneme_dict_path=tts_dataset_files/cmudict-0.7b_nv22.01 \
  heteronyms_path=tts_dataset_files/heteronyms-030921 \
  whitelist_path=tts_dataset_files/lj_speech.tsv \
  exp_manager.exp_dir=./S91_no_mixing \
  +init_from_nemo_model=./tts_en_fastpitch_align.nemo \
  +trainer.max_steps=1000 ~trainer.max_epochs \
  trainer.check_val_every_n_epoch=25 \
  model.train_ds.dataloader_params.batch_size=24 model.validation_ds.dataloader_params.batch_size=24 \
  model.n_speakers=1 model.pitch_mean=121.9 model.pitch_std=23.1 \
  model.pitch_fmin=30 model.pitch_fmax=512 model.optim.lr=2e-4 \
  ~model.optim.sched model.optim.name=adam trainer.devices=1 trainer.strategy=null \
  +model.text_tokenizer.add_blank_at=true \
)

[NeMo W 2022-07-18 15:07:17 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-07-18 15:07:18 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo I 2022-07-18 15:07:19 exp_manager:287] Experiments will be logged at


Epoch 24: 100%|██████████| 26/26 [00:20<00:00,  1.25it/s, loss=4.36, v_num=7-19]Exception ignored in: <function _ConnectionBase.__del__ at 0x7fc71d931c10>
Traceback (most recent call last):
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 132, in __del__
    self._close()
  File "/home/dgxuser/anaconda3/envs/test4/lib/python3.8/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Epoch 24: 100%|██████████| 26/26 [00:21<00:00,  1.19it/s, loss=4.36, v_num=7-19]
                                                                                Epoch 24, global step 625: 'v_loss' reached 1.78086 (best 1.78086), saving model to '/home/dgxuser/Desktop/kailin/S91_no_mixing/FastPitch/2022-07-18_15-07-19/checkpoints/FastPitch--v_loss=1.7809-epoch=24.ckpt' as top 3
Epoch 39: 100%|██████████| 25/25 [00:12<00:00,  1.92it/s, loss=4.24, v_num=7-19]


In [55]:
speaker_id = 91
mixing = False
last_ckpt = get_best_ckpt_from_last_run("./", speaker_id, mixing)
print(last_ckpt)
spec_model = FastPitchModel.load_from_checkpoint(last_ckpt)
spec_model.eval().cuda()

vocoder = HifiGanModel.from_pretrained("tts_hifigan")
vocoder = vocoder.eval().cuda()

S91_no_mixing/FastPitch/2022-07-18_15-07-19/checkpoints/FastPitch--v_loss=1.7809-epoch=24-last.ckpt
[NeMo I 2022-07-18 15:18:30 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2022-07-18 15:19:00 g2ps:86] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2022-07-18 15:19:00 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /home/dgxuser/Desktop/kailin/CHANNEL0/manifest_S91_clean_train.json
      sample_rate: 22050
      sup_data_path: ./fastpitch_sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
     

[NeMo I 2022-07-18 15:19:00 features:200] PADDING: 1
[NeMo I 2022-07-18 15:19:01 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.
[NeMo I 2022-07-18 15:19:01 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo
[NeMo I 2022-07-18 15:19:01 common:789] Instantiating model from pre-trained checkpoint


[NeMo W 2022-07-18 15:19:04 modelPT:149] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2022-07-18 15:19:04 modelPT:156] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2022-07-18 15:19:04 features:200] PADDING: 0


[NeMo W 2022-07-18 15:19:04 features:177] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-07-18 15:19:04 features:200] PADDING: 0
[NeMo I 2022-07-18 15:19:06 save_restore_connector:243] Model HifiGanModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.11.0rc0/tts_hifigan/e6da322f0f7e7dcf3f1900a9229a7e69/tts_hifigan.nemo.


In [56]:
text_dir = '/home/dgxuser/Desktop/kailin/TTS_script_test.txt'
audio_dir = '/home/dgxuser/Desktop/kailin/standardEL_audios/S91_cleanBefore/'
manifest_path = '/home/dgxuser/Desktop/kailin/TTS_test_manifestS' + str(speaker_id) + '_cleanBefore.json'

n = 0
with open(manifest_path, 'w') as fout: 
  with open(text_dir, "r") as f: 
    while True: 
      text = f.readline().strip()
      if text == '': 
        break 
      spec, audio = infer(spec_model, vocoder, text)
      audio = ipd.Audio(audio, rate=22050)
      audio = AudioSegment(audio.data)
      file_dir = audio_dir + str(n) + '.wav'
      audio.export(file_dir, format="wav")

      duration = librosa.core.get_duration(filename=file_dir)
      metadata = {
        "audio_filepath": file_dir,
        "duration": duration, 
        "text": text
      }
      json.dump(metadata, fout)
      fout.write('\n')   
      n = n + 1 

In [57]:
transcript = []
predicted = []

with open(manifest_path, 'r') as f: 
    line = f.readline()
    while line != '': 
        data = json.loads(line)
        transcript.append(data["text"])
        files = [data["audio_filepath"]]
        predicted.append(quartznet.transcribe(paths2audio_files=files)[0])
        line = f.readline()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 0.23821124064729424
